In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
import psycopg2
import pickle
import datetime as dt
import json
import requests
pd.set_option('display.max_columns', 500)

## - Query database
- Looking for the trips table, where the route is the 46A, and the direction is 1.

In [3]:
host = 'localhost'
passwd = 'DBT13GP2020'
user = 'student'
database = 'dublinBus'
port = 5432

In [4]:
# Dataframe of all the trips taken by the 46A going in direction 1.
conn = psycopg2.connect(dbname=database, host=host, port=port, user=user, password=passwd)
cursor = conn.cursor()
df = pd.read_sql_query("SELECT * FROM trips WHERE lineid='46A' AND direction = 1", conn)
conn.close()

In [5]:
df.head()

,datasource,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,basin,tenderlot,suppressed,justificationid,lastupdate,note
0,DB,2018-02-16,6248166,46A,46A_71,1,61495,59400,61082.0,59362.0,BasDef,None,None,None,2018-02-26 12:19:30,",3063561,"
1,DB,2018-02-16,6256926,46A,46A_74,1,67837,63360,68538.0,63343.0,BasDef,None,None,None,2018-02-26 12:19:30,",3064931,"
2,DB,2018-02-16,6247134,46A,46A_74,1,70237,65760,69506.0,65768.0,BasDef,None,None,None,2018-02-26 12:19:30,",3066811,"
3,DB,2018-02-16,6246737,46A,46A_74,1,70717,66240,71578.0,66263.0,BasDef,None,None,None,2018-02-26 12:19:30,",2962749,"
4,DB,2018-02-16,6261883,46A,46A_74,1,69277,64800,69660.0,64967.0,BasDef,None,None,None,2018-02-26 12:19:30,",3067056,"


In [6]:
df.shape

(38745, 16)

In [7]:
# Replace missing actualtime_dep values with plannedtime_dep.
df.actualtime_dep.fillna(df.plannedtime_dep, inplace=True)
df.head()

,datasource,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,basin,tenderlot,suppressed,justificationid,lastupdate,note
0,DB,2018-02-16,6248166,46A,46A_71,1,61495,59400,61082.0,59362.0,BasDef,None,None,None,2018-02-26 12:19:30,",3063561,"
1,DB,2018-02-16,6256926,46A,46A_74,1,67837,63360,68538.0,63343.0,BasDef,None,None,None,2018-02-26 12:19:30,",3064931,"
2,DB,2018-02-16,6247134,46A,46A_74,1,70237,65760,69506.0,65768.0,BasDef,None,None,None,2018-02-26 12:19:30,",3066811,"
3,DB,2018-02-16,6246737,46A,46A_74,1,70717,66240,71578.0,66263.0,BasDef,None,None,None,2018-02-26 12:19:30,",2962749,"
4,DB,2018-02-16,6261883,46A,46A_74,1,69277,64800,69660.0,64967.0,BasDef,None,None,None,2018-02-26 12:19:30,",3067056,"


In [8]:
# Remove rows with missing values for actualtime_arr.
df = df[pd.notnull(df.actualtime_arr)]

In [9]:
# Create a column for trip duration.
df['TRIP_DURATION'] = df.actualtime_arr - df.actualtime_dep

In [10]:
# Create a new column with the hour of the day that the trip began.
df['ACTUALTIME_DEP_H'] = round(df.actualtime_dep/3600)

In [11]:
# Hour of actual time arrival
df['ACTUALTIME_ARR_H'] = round(df.actualtime_arr/3600)

In [12]:
# Average hour of the day of the journey
# Say a trip begins at 1 and ends at 2:
# The journey can be said to have taken place in and around 1:30.
df['AVERAGE_H'] = (df.ACTUALTIME_DEP_H + df.ACTUALTIME_ARR_H) / 2
df.head()

,datasource,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,basin,tenderlot,suppressed,justificationid,lastupdate,note,TRIP_DURATION,ACTUALTIME_DEP_H,ACTUALTIME_ARR_H,AVERAGE_H
0,DB,2018-02-16,6248166,46A,46A_71,1,61495,59400,61082.0,59362.0,BasDef,None,None,None,2018-02-26 12:19:30,",3063561,",1720.0,16.0,17.0,16.5
1,DB,2018-02-16,6256926,46A,46A_74,1,67837,63360,68538.0,63343.0,BasDef,None,None,None,2018-02-26 12:19:30,",3064931,",5195.0,18.0,19.0,18.5
2,DB,2018-02-16,6247134,46A,46A_74,1,70237,65760,69506.0,65768.0,BasDef,None,None,None,2018-02-26 12:19:30,",3066811,",3738.0,18.0,19.0,18.5
3,DB,2018-02-16,6246737,46A,46A_74,1,70717,66240,71578.0,66263.0,BasDef,None,None,None,2018-02-26 12:19:30,",2962749,",5315.0,18.0,20.0,19.0
4,DB,2018-02-16,6261883,46A,46A_74,1,69277,64800,69660.0,64967.0,BasDef,None,None,None,2018-02-26 12:19:30,",3067056,",4693.0,18.0,19.0,18.5


In [13]:
df.AVERAGE_H = df.AVERAGE_H.astype(int)

In [14]:
df['UNIXTIME'] = df[['dayofservice']].apply(lambda x: x[0].timestamp(), axis=1).astype(int)

In [15]:
# Creating column for the day and time of service to join with 
# same/similar column on the historical weather data table
df['UNIX'] = df.UNIXTIME + df.AVERAGE_H * 3600
df.UNIX

0        1518796800
1        1518804000
2        1518804000
3        1518807600
4        1518804000
5        1518804000
6        1518804000
7        1518804000
8        1539734400
9        1531087200
10       1531090800
11       1531087200
12       1531083600
13       1531083600
14       1531090800
15       1531090800
16       1531087200
17       1531087200
18       1531054800
19       1531054800
20       1531062000
21       1531054800
22       1531058400
23       1531058400
24       1531058400
25       1531058400
26       1531058400
27       1531058400
28       1531054800
29       1531062000
            ...    
38712    1537462800
38713    1537466400
38714    1537466400
38715    1537466400
38717    1537470000
38718    1537459200
38720    1537459200
38721    1537455600
38722    1537459200
38723    1546275600
38724    1546272000
38726    1546272000
38727    1546272000
38728    1546196400
38729    1546200000
38730    1546196400
38731    1546200000
38732    1546196400
38733    1546196400


In [16]:
# Removing suppressed rows where suppressed=1.0
df = df.query('suppressed != 1.0')

In [17]:
df.index = range(len(df))

In [18]:
# Creating columns from UNIXTIME for dummies.
df['DAY'] = df['UNIXTIME']
df['MONTH'] = df['UNIXTIME']

In [19]:
# Converting the unix time to datetime format
df.DAY = pd.to_datetime(df['DAY'], unit='s')
df.MONTH = pd.to_datetime(df['MONTH'], unit='s')

In [20]:
# Creating enviroment for dummies
df['DAY'] = df['DAY'].dt.weekday_name
df['MONTH'] = df['MONTH'].dt.month_name()

In [21]:
df.head()

,datasource,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,basin,tenderlot,suppressed,justificationid,lastupdate,note,TRIP_DURATION,ACTUALTIME_DEP_H,ACTUALTIME_ARR_H,AVERAGE_H,UNIXTIME,UNIX,DAY,MONTH
0,DB,2018-02-16,6248166,46A,46A_71,1,61495,59400,61082.0,59362.0,BasDef,None,None,None,2018-02-26 12:19:30,",3063561,",1720.0,16.0,17.0,16,1518739200,1518796800,Friday,February
1,DB,2018-02-16,6256926,46A,46A_74,1,67837,63360,68538.0,63343.0,BasDef,None,None,None,2018-02-26 12:19:30,",3064931,",5195.0,18.0,19.0,18,1518739200,1518804000,Friday,February
2,DB,2018-02-16,6247134,46A,46A_74,1,70237,65760,69506.0,65768.0,BasDef,None,None,None,2018-02-26 12:19:30,",3066811,",3738.0,18.0,19.0,18,1518739200,1518804000,Friday,February
3,DB,2018-02-16,6246737,46A,46A_74,1,70717,66240,71578.0,66263.0,BasDef,None,None,None,2018-02-26 12:19:30,",2962749,",5315.0,18.0,20.0,19,1518739200,1518807600,Friday,February
4,DB,2018-02-16,6261883,46A,46A_74,1,69277,64800,69660.0,64967.0,BasDef,None,None,None,2018-02-26 12:19:30,",3067056,",4693.0,18.0,19.0,18,1518739200,1518804000,Friday,February


In [22]:
# set up dummies features
df_day_dummies = pd.get_dummies(df.DAY)
df_month_dummies = pd.get_dummies(df.MONTH)

In [23]:
df_hour_dummies = pd.get_dummies(df.AVERAGE_H)

In [24]:
df.shape

(37084, 24)

In [25]:
trips_df = pd.concat([df, df_hour_dummies, df_day_dummies, df_month_dummies], axis=1, join_axes=[df.index])

In [26]:
# ## Guidance Dataframe to check how many buses go every hour on average on a particular day
# filterinfDataframe = trips_df[(trips_df['AVERAGE_H'] == 16) & (trips_df['dayofservice'] == '2018-02-16') ]
# filterinfDataframe

## - Join weather table with trips table above based upon time.

In [27]:
conn = psycopg2.connect(dbname=database, host=host, port=port, user=user, password=passwd)
cursor = conn.cursor()
# Historical weather data from database
weather_df = pd.read_sql_query('SELECT * FROM historical_weather', conn)
conn.close()
weather_df.head()

,dt,dt_iso,timezone,city_name,lat,lng,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1514764800,2018-01-01 00:00:00,0,Dublin,53.349805,-6.26031,4.15,-6.49,3.84,5.79,990,None,None,87,12.86,240,NaN,NaN,None,NaN,40,520,Rain,light intensity shower rain,09n
1,1514768400,2018-01-01 01:00:00,0,Dublin,53.349805,-6.26031,4.14,-5.79,3.65,5.86,991,None,None,87,11.83,240,NaN,NaN,None,NaN,75,520,Rain,light intensity shower rain,09n
2,1514772000,2018-01-01 02:00:00,0,Dublin,53.349805,-6.26031,4.61,-5.77,3.85,5.99,990,None,None,81,12.35,240,NaN,NaN,None,NaN,40,802,Clouds,scattered clouds,03n
3,1514775600,2018-01-01 03:00:00,0,Dublin,53.349805,-6.26031,4.64,-5.73,4.00,6.14,990,None,None,81,12.35,240,NaN,NaN,None,NaN,40,802,Clouds,scattered clouds,03n
4,1514779200,2018-01-01 04:00:00,0,Dublin,53.349805,-6.26031,5.04,-4.91,4.11,6.22,990,None,None,81,11.83,240,NaN,NaN,None,NaN,40,802,Clouds,scattered clouds,03n


In [28]:
weather_df['MONTH'] = weather_df['dt']
weather_df.MONTH = pd.to_datetime(weather_df['MONTH'], unit='s')
weather_df['MONTH'] = weather_df['MONTH'].dt.month

In [29]:
weather_df.head()

,dt,dt_iso,timezone,city_name,lat,lng,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,MONTH
0,1514764800,2018-01-01 00:00:00,0,Dublin,53.349805,-6.26031,4.15,-6.49,3.84,5.79,990,None,None,87,12.86,240,NaN,NaN,None,NaN,40,520,Rain,light intensity shower rain,09n,1
1,1514768400,2018-01-01 01:00:00,0,Dublin,53.349805,-6.26031,4.14,-5.79,3.65,5.86,991,None,None,87,11.83,240,NaN,NaN,None,NaN,75,520,Rain,light intensity shower rain,09n,1
2,1514772000,2018-01-01 02:00:00,0,Dublin,53.349805,-6.26031,4.61,-5.77,3.85,5.99,990,None,None,81,12.35,240,NaN,NaN,None,NaN,40,802,Clouds,scattered clouds,03n,1
3,1514775600,2018-01-01 03:00:00,0,Dublin,53.349805,-6.26031,4.64,-5.73,4.00,6.14,990,None,None,81,12.35,240,NaN,NaN,None,NaN,40,802,Clouds,scattered clouds,03n,1
4,1514779200,2018-01-01 04:00:00,0,Dublin,53.349805,-6.26031,5.04,-4.91,4.11,6.22,990,None,None,81,11.83,240,NaN,NaN,None,NaN,40,802,Clouds,scattered clouds,03n,1


In [30]:
weather_df.rename(columns={'dt' : 'UNIX'}, inplace=True)

In [31]:
combine_df = pd.merge(trips_df, weather_df, on=['UNIX'])

In [32]:
combine_df.head()

,datasource,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,basin,tenderlot,suppressed,justificationid,lastupdate,note,TRIP_DURATION,ACTUALTIME_DEP_H,ACTUALTIME_ARR_H,AVERAGE_H,UNIXTIME,UNIX,DAY,MONTH_x,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,April,August,December,February,January,July,June,March,May,November,October,September,dt_iso,timezone,city_name,lat,lng,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,MONTH_y
0,DB,2018-02-16,6248166,46A,46A_71,1,61495,59400,61082.0,59362.0,BasDef,None,None,None,2018-02-26 12:19:30,",3063561,",1720.0,16.0,17.0,16,1518739200,1518796800,Friday,February,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2018-02-16 16:00:00,0,Dublin,53.349805,-6.26031,7.5,-0.9,6.72,8.0,1016,None,None,61,9.26,190,NaN,NaN,None,NaN,75,803,Clouds,broken clouds,04d,2
1,DB,2018-02-16,6254484,46A,46A_74,1,61335,56640,61631.0,56579.0,BasDef,None,None,None,2018-02-26 12:19:30,",3058212,",5052.0,16.0,17.0,16,1518739200,1518796800,Friday,February,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2018-02-16 16:00:00,0,Dublin,53.349805,-6.26031,7.5,-0.9,6.72,8.0,1016,None,None,61,9.26,190,NaN,NaN,None,NaN,75,803,Clouds,broken clouds,04d,2
2,DB,2018-02-16,6259474,46A,46A_74,1,61815,57120,62287.0,57067.0,BasDef,None,None,None,2018-02-26 12:19:30,",3064343,",5220.0,16.0,17.0,16,1518739200,1518796800,Friday,February,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2018-02-16 16:00:00,0,Dublin,53.349805,-6.26031,7.5,-0.9,6.72,8.0,1016,None,None,61,9.26,190,NaN,NaN,None,NaN,75,803,Clouds,broken clouds,04d,2
3,DB,2018-02-16,6247413,46A,46A_74,1,60855,56160,61213.0,56284.0,BasDef,None,None,None,2018-02-26 12:19:30,",3061722,",4929.0,16.0,17.0,16,1518739200,1518796800,Friday,February,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2018-02-16 16:00:00,0,Dublin,53.349805,-6.26031,7.5,-0.9,6.72,8.0,1016,None,None,61,9.26,190,NaN,NaN,None,NaN,75,803,Clouds,broken clouds,04d,2
4,DB,2018-02-16,6251904,46A,46A_74,1,62295,57600,62675.0,57665.0,BasDef,None,None,None,2018-02-26 12:19:30,",3063193,",5010.0,16.0,17.0,16,1518739200,1518796800,Friday,February,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2018-02-16 16:00:00,0,Dublin,53.349805,-6.26031,7.5,-0.9,6.72,8.0,1016,None,None,61,9.26,190,NaN,NaN,None,NaN,75,803,Clouds,broken clouds,04d,2


In [33]:
combine_df = combine_df[[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                         17, 18, 19, 20, 21, 22, 23, 24,
                         'Friday', 'Monday', 'Saturday', 
                         'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 
                         'April', 'August', 'December', 'February',
                         'January', 'July', 'June', 'March', 'May',
                         'November', 'October', 'September',
                         'temp', 'humidity', 'clouds_all', 
                         'TRIP_DURATION']]

In [34]:
# Trip duration is in seconds, convert to minutes and round to the nearest integer
combine_df['TRIP_DURATION'] = round(combine_df['TRIP_DURATION']/60)

In [35]:
combine_df['TRIP_DURATION'] = combine_df['TRIP_DURATION'].astype(int)

In [36]:
combine_df.head()

,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,April,August,December,February,January,July,June,March,May,November,October,September,temp,humidity,clouds_all,TRIP_DURATION
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75,29
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75,84
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75,87
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75,82
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75,84


In [37]:
combine_df.shape

(37211, 43)

## - Table shape preparation

In [38]:
# Assign data from first four columns to X variable
X = combine_df.iloc[:, 0:42]

# Assign data from fifth column to y variable
y = combine_df['TRIP_DURATION']

In [39]:
X.head()

,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,April,August,December,February,January,July,June,March,May,November,October,September,temp,humidity,clouds_all
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75


In [40]:
y.head()

0    29
1    84
2    87
3    82
4    84
Name: TRIP_DURATION, dtype: int64

## - Random Forest

In [41]:
regr = RandomForestRegressor(n_estimators=100, max_depth=3, random_state=0)
regr.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [42]:
# storing the model trained on the full data set to a pickle file
pkl_filename = "RANDOM_FOREST_2018_46A_1.pkl"
with open(pkl_filename, 'wb') as file:  
    pickle.dump(regr, file)

In [43]:
# Compute the importance of each feature based on the model
pd.DataFrame({'feature': X.columns, 'importance': regr.feature_importances_})

,feature,importance
0,5,0.000000
1,6,0.000000
2,7,0.000000
3,8,0.000000
4,9,0.000000
5,10,0.000000
6,11,0.000000
7,12,0.000000
8,13,0.000000
9,14,0.000000


In [44]:
X.head()

,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,April,August,December,February,January,July,June,March,May,November,October,September,temp,humidity,clouds_all
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75


In [45]:
y.head()

0    29
1    84
2    87
3    82
4    84
Name: TRIP_DURATION, dtype: int64

In [46]:
# predict for 4 pm on a Friday in February with 10.8 TEMP, 71 HUMIDITY and 75 CLOUDS_ALL
print("This journey is predicted to take", 
      round(regr.predict([[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0, 
                           1, 0, 0, 0, 0, 0, 0,
                           0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 
                           7.5, 61, 75]])[0]),"minutes")

This journey is predicted to take 73.0 minutes


In [47]:
pred = regr.predict(X)

In [48]:
predictions = pd.DataFrame(pred)
predictions.rename(columns={0:'TIME_ESTIMATE'}, inplace=True )
predictions['TIME_ESTIMATE'] = round(predictions['TIME_ESTIMATE'])
predictions['TIME_ESTIMATE'] = predictions['TIME_ESTIMATE'].astype(int)
predictions.head(50)

,TIME_ESTIMATE
0,73
1,73
2,73
3,73
4,73
5,73
6,73
7,73
8,73
9,73


In [49]:
df_compare = pd.concat([X, y, predictions], axis=1, join_axes=[df.index])
df_compare.head(10)

,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,April,August,December,February,January,July,June,March,May,November,October,September,temp,humidity,clouds_all,TRIP_DURATION,TIME_ESTIMATE
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.50,61,75,29,73
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.50,61,75,84,73
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.50,61,75,87,73
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.50,61,75,82,73
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.50,61,75,84,73
5,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.50,61,75,81,73
6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.50,61,75,78,73
7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.50,61,75,70,73
8,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,6.43,66,75,87,73
9,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,6.43,66,75,62,73


In [50]:
print("Accuracy: ", metrics.accuracy_score(y, predictions))
# Include more metrics here.

Accuracy:  0.032436645078068316


## - Train Test Split

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# print("Training data:\n", pd.concat([X_train, y_train], axis=1))
# print("\nTest data:\n", pd.concat([X_test, y_test], axis=1))

## - Picklling!

In [52]:
# Load the pickle file
with open('RANDOM_FOREST_2018_46A_1.pkl','rb') as pickle_file:
        rfr_ = pickle.load(pickle_file)

In [53]:
X.head()

,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,April,August,December,February,January,July,June,March,May,November,October,September,temp,humidity,clouds_all
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7.5,61,75


## - Build an input dataframe for testing.

In [54]:
# Generate current date and time
day = dt.datetime.today().weekday() # current day of the week 0=Monday
hour = (dt.datetime.now().hour)  # current hour 
month = (dt.datetime.now().month)

In [55]:
print(month)
month = 1
print(month)

7
1


In [56]:
# Dataframe to hold current time
df_times = pd.DataFrame({hour: [hour]})

In [57]:
df_times

,18
0,18


In [58]:
df_times['5'] = np.where((df_times[hour] == 5), 1, 0)
df_times['6'] = np.where((df_times[hour] == 6), 1, 0)
df_times['7'] = np.where((df_times[hour] == 7), 1, 0)
df_times['8'] = np.where((df_times[hour] == 8), 1, 0)
df_times['9'] = np.where((df_times[hour] == 9), 1, 0)
df_times['10'] = np.where((df_times[hour] == 10), 1, 0)
df_times['11'] = np.where((df_times[hour] == 11), 1, 0)
df_times['12'] = np.where((df_times[hour] == 12), 1, 0)
df_times['13'] = np.where((df_times[hour] == 13), 1, 0)
df_times['14'] = np.where((df_times[hour] == 14), 1, 0)
df_times['15'] = np.where((df_times[hour] == 15), 1, 0)
df_times['16'] = np.where((df_times[hour] == 16), 1, 0)
df_times['17'] = np.where((df_times[hour] == 17), 1, 0)
df_times['18'] = np.where((df_times[hour] == 18), 1, 0)
df_times['19'] = np.where((df_times[hour] == 19), 1, 0)
df_times['20'] = np.where((df_times[hour] == 20), 1, 0)
df_times['21'] = np.where((df_times[hour] == 21), 1, 0)
df_times['22'] = np.where((df_times[hour] == 22), 1, 0)
df_times['23'] = np.where((df_times[hour] == 23), 1, 0)
df_times['24'] = np.where((df_times[hour] == 24), 1, 0)

In [59]:
df_times = df_times.drop(df_times.columns[0], axis=1)

In [60]:
# Dataframe for the days of the week
df_days = pd.DataFrame({day: [day]})

In [61]:
df_days

,0
0,0


In [62]:
# Create the dummy columns for all of the days.
df_days['Friday'] = np.where((df_days[day] == 4), 1, 0)
df_days['Monday'] = np.where((df_days[day] == 0), 1, 0)
df_days['Saturday'] = np.where((df_days[day] == 5), 1, 0)
df_days['Sunday'] = np.where((df_days[day] == 6), 1, 0)
df_days['Thursday'] = np.where((df_days[day] == 3), 1, 0)
df_days['Tuesday'] = np.where((df_days[day] == 1), 1, 0)
df_days['Wednesday'] = np.where((df_days[day] == 2), 1, 0)

In [63]:
df_days = df_days.drop(df_days.columns[0], axis=1)

In [64]:
df_days.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,1,0,0,0,0,0


In [65]:
df_month = pd.DataFrame({month: [month]})

In [66]:
# Create the dummy columns for all of the months.
df_month['April'] = np.where((df_month[month] == 4), 1, 0)
df_month['August'] = np.where((df_month[month] == 8), 1, 0)
df_month['December'] = np.where((df_month[month] == 12), 1, 0)
df_month['February'] = np.where((df_month[month] == 2), 1, 0)
df_month['January'] = np.where((df_month[month] == 1), 1, 0)
df_month['July'] = np.where((df_month[month] == 7), 1, 0)
df_month['June'] = np.where((df_month[month] == 6), 1, 0)
df_month['March'] = np.where((df_month[month] == 3), 1, 0)
df_month['May'] = np.where((df_month[month] == 5), 1, 0)
df_month['November'] = np.where((df_month[month] == 11), 1, 0)
df_month['October'] = np.where((df_month[month] == 10), 1, 0)
df_month['September'] = np.where((df_month[month] == 9), 1, 0)

In [67]:
df_month = df_month.drop(df_month.columns[0], axis=1)
df_month

,April,August,December,February,January,July,June,March,May,November,October,September
0,0,0,0,0,1,0,0,0,0,0,0,0


In [68]:
# Call to the weather api for the current weather conditions
WEATHER_URI = "http://api.openweathermap.org/data/2.5/weather"
weather_api = "0af2c4378e1bfb001a3e457cc32410be"
response = requests.get(WEATHER_URI, params={"id": 2964574, "appid": weather_api})
kelvin = 273.15
# parse the data
data = response.text
parsed = json.loads(data)
# create dataframes
temp_df = pd.DataFrame(parsed['main'], index=[0])
temp_df.temp[0] = temp_df.temp[0] - kelvin
cloud_df = pd.DataFrame(parsed['clouds'], index=[0])
cloud_df = cloud_df.rename(columns={'all': 'clouds_all'})

In [69]:
# Dataframe to hold correct weather input for model
df_weather = pd.DataFrame({'temp': temp_df['temp'], 'humidity': temp_df['humidity'], 'clouds_all': cloud_df['clouds_all']})

In [70]:
# Create an input dataframe - combining each of the above
dfX = pd.concat([df_times, df_days, df_month, df_weather], axis =1)
dfX

,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,April,August,December,February,January,July,June,March,May,November,October,September,temp,humidity,clouds_all
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.11,63,75


In [71]:
# Predict journey time (first to last stop) given local time and weather
print("This entire journey will take around", int(round(rfr_.predict(dfX)[0])), "minutes.")

This entire journey will take around 73 minutes.


In [72]:
## Guidance Dataframe to check how many buses go every hour on average on a particular day
filterinfDataframe = combine_df[(combine_df[12] == 1) 
                              & (combine_df['July'] == 1)
                               & (combine_df['July'] == 1)]
filterinfDataframe.head(10)

,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,April,August,December,February,January,July,June,March,May,November,October,September,temp,humidity,clouds_all,TRIP_DURATION
1227,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,17.97,73,75,63
1228,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,17.97,73,75,88
1229,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,17.97,73,75,70
1230,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,17.97,73,75,65
1231,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,17.97,73,75,80
3622,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,17.25,56,75,72
3623,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,17.25,56,75,76
3624,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,17.25,56,75,81
3625,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,17.25,56,75,68
3626,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,17.25,56,75,76


## Set up the correct sequence of stops next. 
- Come up with a way to get the count value for the sequence of stops. 

In [74]:
# Obtaining the sequence of stops for this route in this direction through query on one tripid.
# Soundness of this is based on the presumption that all other tripids associated
# with this route and going in this direction have the exact same sequence of stops.
conn = psycopg2.connect(dbname=database, host=host, port=port, user=user, password=passwd)
cursor = conn.cursor()
sequence_df = pd.read_sql_query("SELECT * FROM leavetimes WHERE tripid='5958395' ORDER BY progrnumber ASC", conn)
conn.close()

In [75]:
sequence_df.head(60)

,datasource,dayofservice,tripid,progrnumber,stoppointid,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,vehicleid,passengers,passengersin,passengersout,distance,suppressed,justificationid,lastupdate,note
0,DB,2018-01-01,5958395,1,807,64800,64800,64884,64884,2406878,None,None,None,None,None,None,2018-01-08 17:21:10,None
1,DB,2018-01-01,5958395,2,808,64887,64887,64916,64937,2406878,None,None,None,None,None,None,2018-01-08 17:21:10,None
2,DB,2018-01-01,5958395,3,809,64938,64938,64959,64991,2406878,None,None,None,None,None,None,2018-01-08 17:21:10,None
3,DB,2018-01-01,5958395,4,810,65019,65019,65022,65022,2406878,None,None,None,None,None,None,2018-01-08 17:21:10,None
4,DB,2018-01-01,5958395,5,811,65070,65070,65055,65068,2406878,None,None,None,None,None,None,2018-01-08 17:21:10,None
5,DB,2018-01-01,5958395,6,812,65101,65101,65088,65088,2406878,None,None,None,None,None,None,2018-01-08 17:21:10,None
6,DB,2018-01-01,5958395,7,813,65137,65137,65105,65105,2406878,None,None,None,None,None,None,2018-01-08 17:21:10,None
7,DB,2018-01-01,5958395,8,814,65211,65211,65138,65144,2406878,None,None,None,None,None,None,2018-01-08 17:21:10,None
8,DB,2018-01-01,5958395,9,81,65293,65293,65196,65215,2406878,None,None,None,None,None,None,2018-01-08 17:21:10,None
9,DB,2018-01-01,5958395,10,817,65370,65370,65251,65251,2406878,None,None,None,None,None,None,2018-01-08 17:21:10,None


In [82]:
sequence_df.stoppointid

0      807
1      808
2      809
3      810
4      811
5      812
6      813
7      814
8       81
9      817
10     818
11     819
12     264
13    6059
14     334
15     406
16     747
17     842
18     845
19     846
20     847
21     848
22    2795
23     756
24     757
25     758
26     759
27     760
28     761
29     762
30     763
31    2007
32    2008
33    2009
34    2010
35     435
36    7353
37    4571
38    2013
39    2014
40    4636
41    2016
42    2015
43    2017
44    2018
45    2019
46    2020
47    2021
48    2022
49    4565
50    4566
51    4567
52    2031
53    2032
54    2033
55    2034
56    2035
57    2036
58    2039
Name: stoppointid, dtype: int64

In [83]:
sequence_df.stoppointid.to_json(r'./46A_dir1_stops.json', orient='records')

In [ ]:
total_number_of_stops = len(sequence_df)

In [ ]:
# Choose progrnumbers for origin and desintation locations
x = 1
y = 17

In [ ]:
origin_stop = sequence_df.query("progrnumber==("+str(x)+")")['progrnumber'][x-1]
destination_stop = sequence_df.query("progrnumber==("+str(y)+")")['progrnumber'][y-1]

In [ ]:
number_of_stops_travelled = destination_stop - origin_stop
percentage_of_route_travelled = round((number_of_stops_travelled/(total_number_of_stops - 1)) * 100)
print("This represents %" + str(percentage_of_route_travelled) + " of the whole journey")

In [ ]:
total_time = int(round((int(round(rfr_.predict(dfX)[0])) * percentage_of_route_travelled) / 100))
print("If you wish to travel between stops",
      sequence_df.query("progrnumber==("+str(x)+")")['stoppointid'][x-1], 
      "and", 
      sequence_df.query("progrnumber==("+str(y)+")")['stoppointid'][y-1], 
      "your journey will take approximately", total_time, "minutes.")